# Generate Painels from the Annotations

```
Author: Phillipe Cardenuto (RECOD - UNICAMP)
email: phillipe.cardenuto@gmail.com
```

This is an auxiliar notebook to generate the ground-truth images used on the Segmenation and Retrieval Tasks

# Instructions
In this notebook we use the bounding box annotations to create a panel mask image for each annotated data and to extracted its panels.

### Enviroment
To use this notebook make sure to have the following configurations installed:
- python 3.6
- opencv-python>=3
- pillow >=7
# Important
Make sure to insert each downloaded figure in the same location as its 'figure-path' annotation in the JSON file

The JSON is structured as following:
``` json
{
 <doi> :
        {
            'figures': 
                        {'fig1':
                                {'figure-path': <dataset-figure-path>,
                                  'panels': 
                                            {'000': {'x0': <int>, 'y0': <int>, 'x1': <int>, 'y1': <int>},
                                             '001': {'x0': <int>, 'y0': <int>, 'x1': <int>, 'y1': <int>},
                                              ....,
                                             'NNN': {'x0': <int>, 'y0': <int>, 'x1': <int>, 'y1': <int>},
                                            }
                                 'groundtruth-path': <dataset-destination-path-gt>,
                                 'figure-url': <original-url>
                                },

                            ...,
                           'figN':   {'figure-path': <dataset-figure-path>,
                                  'panels': 
                                            {'000': {'x0': <int>, 'y0': <int>, 'x1': <int>, 'y1': <int>},
                                             '001': {'x0': <int>, 'y0': <int>, 'x1': <int>, 'y1': <int>},
                                              ....,
                                             'NNN': {'x0': <int>, 'y0': <int>, 'x1': <int>, 'y1': <int>},
                                            }
                                 'groundtruth-path': <dataset-destination-path-gt>,
                                 'figure-url': <original-url>
                                }
                        }
        }
}
```

In [1]:
# Import cell
import os
from glob import glob
import cv2
import numpy as np
from  PIL import Image, ImageDraw 
import json

In [2]:
# Read Json annotation
with open('figure-panel-segmentation.json','r') as js:
    panel_annotations = json.load(js)

In [3]:
def read_image(filename):
    return np.array(Image.open(filename))
def write_image(savename,img):
    # create save dir
    os.makedirs(os.path.dirname(savename),exist_ok=True)
    # write image
    pil_img = Image.fromarray(img)
    pil_img.save(savename)


def create_panel_mask(fig_annotation):
    
    # Open figure
    figure = read_image(fig_annotation['figure-path'])
    height, width= figure.shape[:2]
    
    # Create a image with the same size of figure
    panels_mask = np.array(Image.new("P",(width,height)))
    
    # Mark the region of each panel
    for panel_name, panel in fig_annotation['panels'].items():
        panels_mask[panel['x0']: panel['x1'], panel['y0']: panel['y1']] =  255

    # Save panel_mask
    if not os.path.isdir(os.path.dirname(fig_annotation['groundtruth-path'])):
        os.makedirs(os.path.dirname(fig_annotation['groundtruth-path']))
    # Write Panel Mask
    write_image(fig_annotation['groundtruth-path'], panels_mask)
    
    # Save all cropped panels
    for panel_name, panel in fig_annotation['panels'].items():
        if len(figure.shape) > 2:
            panel_img = figure[panel['x0']: panel['x1'], panel['y0']: panel['y1'],:]
        else:
            panel_img = figure[panel['x0']: panel['x1'], panel['y0']: panel['y1']]
            
        # Get panel save name
        panel_save_name = fig_annotation['figure-path'].replace("figures/",'figures-panels/')
        panel_save_name = panel_save_name[:panel_save_name.rfind(("."))] + f"_{panel_name}.png"
        
        # Write Figure Panel
        write_image(panel_save_name, panel_img)

# Extracts Panels and Create Panels Mask

In [4]:
for doi in panel_annotations.keys():
    for _, fig_annotation in panel_annotations[doi]['figures'].items():
        create_panel_mask(fig_annotation)